3'UTR Python calculations


In [ ]:
from os import listdir
from os.path import isfile, join
import pandas as pd
import scipy.stats
from statsmodels.stats.multitest import fdrcorrection


Function to obtain list of genes (SYMBOL names) from the results of enrichment analysis. 

In [ ]:
def genes_extractor(path):
    all_genes = {}
    with open(path, mode = "r") as data:
        line = data.readline().strip()
        while line !="":
            print(line)
            line = line.split("\t")
            print(line)
            genes = line[8].split("/")
            print(genes)
            for i in genes:
                if not i in all_genes:
                    all_genes[i] = ""
            line = data.readline().strip()
    result = list(all_genes.keys())
    print(result)
    return result

In [ ]:
with open("../results/GENES_11_HUM.tsv", mode = "w") as out:
    gene_list_MF = genes_extractor("../results/hum_11_50_go_zer_ribo.txt")
    for i in gene_list_MF:
        print(i, file = out)

with open("../results/GENES_10_HUM.tsv", mode = "w") as out:
    gene_list_MF = genes_extractor("../results/hum_10_go.tsv")
    for i in gene_list_MF:
        print(i, file = out)

Script to find organisms with small 3'UTR length median values

In [ ]:
dir_path = "PATH TO FOLDER WITH SEPARATE RESULTS"
filenames = [f for f in listdir(dir_path) if isfile(join(dir_path, f))]

result = []
for i in filenames:
    if i.endswith("tsv"):
        data =  pd.read_csv(dir_path+i, delimiter="\t", names = ["Transcript_ID", "Codon", "Length"], usecols = [1,2])
        med = data.groupby(["Codon"]).median()["Length"].tolist()
        count = len(data.index)
        try:
            med_sum = med[0]+med[1]+med[2]
            result.append((i, med[0], med[1], med[2],med_sum, count))
        except IndexError:
            pass
result.sort(key=lambda x:x[4])

Function to statistically test difference between median values via Mood's median test

In [ ]:
def codon_medians (pandas_input):
    uga = pandas_input[pandas_input["Codon"] == "TGA"]["Length"].tolist()
    uaa = pandas_input[pandas_input["Codon"] == "TAA"]["Length"].tolist()
    uag = pandas_input[pandas_input["Codon"] == "TAG"]["Length"].tolist()   
    
    uaa_uga = scipy.stats.median_test(uaa, uga).pvalue
    uaa_uag = scipy.stats.median_test(uaa, uag).pvalue
    uag_uga = scipy.stats.median_test(uag, uga).pvalue

    print(f"UAA-UAG = {uaa_uag}")
    print(f"UAA-UGA = {uaa_uga}")
    print(f"UAG-UGA = {uag_uga}")
    print(fdrcorrection([uaa_uag, uaa_uga, uag_uga]))

    return None

In [ ]:
human_data = pd.read_csv("../data/RES_Homo_sapiens.GRCh38.107.utrs.tsv", sep = "\t", names = ["Transcript_ID", "Codon", "Length"])
codon_medians(human_data)
#The same procedure is for other organisms and total table (../data/Org_tables/TOTAL_{i}.zip) which previously had to be unzipped and merged

In [ ]:
#All human data vs human housekeeping genes
human_house = pd.read_csv("../data/human_house.tsv", sep = "\t")
uga_house = human_house[human_house["Codon"] == "UGA"]["Length"].tolist()
uaa_house = human_house[human_house["Codon"] == "UAA"]["Length"].tolist()
uag_house = human_house[human_house["Codon"] == "UAG"]["Length"].tolist() 

uga_human = human_data[human_data["Codon"] == "TGA"]["Length"].tolist()
uaa_human = human_data[human_data["Codon"] == "TAA"]["Length"].tolist()
uag_human = human_data[human_data["Codon"] == "TAG"]["Length"].tolist()

uaa_uaa = scipy.stats.median_test(uaa_human, uaa_house).pvalue
uag_uag = scipy.stats.median_test(uag_human, uag_house).pvalue
uga_uga = scipy.stats.median_test(uga_human, uga_house).pvalue

print(f"UAA = {uaa_uaa}")
print(f"UGA = {uga_uga}")
print(f"UAG = {uag_uag}")
print(fdrcorrection([uaa_uaa, uga_uga, uag_uag]))

In [ ]:
rabbit_data = pd.read_csv("../data/RABORTH.tsv", sep = "\t")
codon_medians(rabbit_data)

In [ ]:
#Mood's median test for two groups of enrichment analysis
hum_50 = pd.read_csv("../results/hum_50.tsv", sep = "\t")
hum_10 = pd.read_csv("../results/hum_10.tsv", sep = "\t")

print("11-50")
codon_medians(hum_50)

In [ ]:
uga = hum_10[(hum_10["Codon"] == "UGA") & (hum_10["Length"]>0)]["Length"]
uaa = hum_10[(hum_10["Codon"] == "UAA") & (hum_10["Length"]>0)]["Length"]
uag = hum_10[(hum_10["Codon"] == "UAG") & (hum_10["Length"]>0)]["Length"]   

uaa_uga = scipy.stats.median_test(uaa, uga).pvalue
uaa_uag = scipy.stats.median_test(uaa, uag).pvalue
uag_uga = scipy.stats.median_test(uag, uga).pvalue
all_tog = scipy.stats.median_test(uag, uga,uaa).pvalue

print(f"UAA-UAG = {uaa_uag}")
print(f"UAA-UGA = {uaa_uga}")
print(f"UAG-UGA = {uag_uga}")
print(fdrcorrection([uaa_uag, uaa_uga, uag_uga]))

In [ ]:
o_lycim = pd.read_csv("../data/RES_Ostreococcus_lucimarinus.ASM9206v1.54.utrs.tsv", sep="\t", names = ["Transcript_ID", "Codon", "Length"])
a_lyrat = pd.read_csv("../data/RES_Arabidopsis_lyrata.v.1.0.54.utrs.tsv", sep="\t", names = ["Transcript_ID", "Codon", "Length"])
d_willis = pd.read_csv("../data/RES_Drosophila_willistoni.dwil_caf1.54.utrs.tsv", sep="\t", names = ["Transcript_ID", "Codon", "Length"])
p_perni = pd.read_csv("../data/RES_Phlebotomus_perniciosus_gca918844115.pperniciosus_illum_n82538_165Mb.54.utrs.tsv", sep="\t", names = ["Transcript_ID", "Codon", "Length"])

print("o_lycim")
codon_medians(o_lycim)
print("a_lyrat")
codon_medians(a_lyrat)
print("d_willis")
codon_medians(d_willis)
print("p_perni")
codon_medians(p_perni)
